## Youtube video search PoC

In [1]:
import requests
from bs4 import BeautifulSoup
import re
import json
import pytube
import os
from pathlib import Path
from moviepy.editor import *

from tqdm.auto import tqdm  # !pip install tqdm

import whisper
import torch  # pytorch install steps: pytorch.org

import pinecone # pip install --upgrade pinecone-client
from sentence_transformers import SentenceTransformer

## Inputs

In [2]:
channel_names = ['@FinancialTimes']
#channel_names = ['UCfkpRl1dykCRyyspOB0hffQ', 'ProjectFarm', 'PaintballOO7', 'StrangeLoopConf', 'FarmCraft101', 'standupmaths', 'NDCConferences', 'TheRoyalInstitution', 'bobreynolds', 'abadcliche', 'flutterdev', 'UCCkL8DHm5KSE5MiDWepOHUw', 'MIT', 'msadaghd', 'HooverInstitution', 'Kurzgesagt', 'UCm9K6rby98W8JigLoZOh6FQ', 'Luapper', 'shanselman', 'destinws2', 'steventhebrave', '1veritasium', 'TheoBrowne1017']
pinecone_api_key = 'c65fa925-08e1-4af0-b08b-1104c6ffba25' # https://app.pinecone.io/projects

youtube_dl_options = {
    'skip_download': True,
    'ignoreerrors': True
}

## Helper functions

In [14]:
def extract_video_ids(text):
    video_ids = []
    pattern = r'(?<=watch\?v=)[\w-]+'
    matches = re.findall(pattern, text)
    for match in matches:
        video_ids.append(match)
    return video_ids

def extract_video_titles(html):
    video_ids = []
    pattern = r'\"title":{"runs":\[{"text":".*?\"'
    matches = re.findall(pattern, html)
    for match in matches:
        #video_ids.append(match)
        
        pattern2 = r'\"text":".*?\"'
        matches2 = re.findall(pattern2, match)
        for match2 in matches2:
            text = match2.replace("\"text\":\"", "")[:-1]
            text = text.replace(":", "")
            text = text.replace(".", "")
            text = text.replace("?", "")
            text = text.replace("'", "")
            text = text.replace("\\", "")
            text = text.replace("|", "")
            text = text.replace(",", "")
            text = text.replace("xe2x80x99", "’")
            video_ids.append(text)
    return video_ids

def download_youtube_video(url, filepath):
    youtube = pytube.YouTube(url)
    video = youtube.streams.get_highest_resolution()
    video.download(filepath)
    
def download_channel_videos(channel_name):
    video_index = {}
    # Create a directory to store the videos
    if not os.path.exists(channel_name):
        os.mkdir(channel_name)
    # Get the HTML content of the channel page
    url = f"https://www.youtube.com/{channel_name}/videos"
    response = requests.get(url)
    if response.status_code != 200:
        print(f"{channel_name} does not exist or has no videos.")
    
        
    html_content = response.content
    
    results = extract_video_ids(str(html_content))
    titles = extract_video_titles(str(html_content))
    count = 0;
    for result in results:
        video_index[titles[count]] = result
        count = count + 1
    
    video_urls = []
    for result in results:
        video_urls.append(f"https://www.youtube.com/watch?v={result}")
        
    for url in video_urls:
        print(url, channel_name)
        youtube = pytube.YouTube(url)
        video = youtube.streams.get_lowest_resolution()
        video.download(channel_name)

    print(f"All videos from {channel_name} have been downloaded.")
    return video_index

def get_mp4_files(directory):
    mp4_files = []
    for file in os.listdir(directory):
        if file.endswith(".mp4"):
            mp4_files.append(os.path.join(directory, file))
    return mp4_files

def save_audio_from_videoIDs(directory):
    video_IDs = get_mp4_files(directory)
    for videoID in tqdm(video_IDs):
        check_file = f"{videoID.replace('.mp4', '.mp3')}"
        file = Path(check_file)
        if file.exists():
            print('skipping file ', check_file)
            continue
        MP4ToMP3(f"{videoID}", f"{videoID.replace('.mp4', '.mp3')}")

def MP4ToMP3(mp4, mp3):
    FILETOCONVERT = AudioFileClip(mp4)
    FILETOCONVERT.write_audiofile(mp3)
    FILETOCONVERT.close()
    
def get_text_from_data(start, end, data):
    text = ""
    for i in range(start,end):
        text += data[i]['text']+' '
    return text

## Use whisper for transcriptions for audio extracted from videos

In [4]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

model = whisper.load_model("small").to(device)

model_id = "multi-qa-mpnet-base-dot-v1"
model_embed = SentenceTransformer(model_id)
dim = model_embed.get_sentence_embedding_dimension()

index_id = "audio"
pinecone.init(
    api_key=pinecone_api_key,  # app.pinecone.io
    environment="us-west1-gcp"
)
if index_id not in pinecone.list_indexes():
    pinecone.create_index(
        index_id,
        dim,
        metric="dotproduct"
    )

index = pinecone.Index(index_id)
index.describe_index_stats()

# we encode and insert in batches of 64
batch_size = 64

cuda


## Write embeddings to pinecone as we process each file

In [ ]:
for channel_name in channel_names:
    try:
        video_index = download_channel_videos(channel_name)
        save_audio_from_videoIDs(channel_name)
    except:
        print("error getting video")

    # get list of MP3 audio files
    paths = [str(x) for x in Path(channel_name).glob('*.mp3')]
    
    transcriptions = []
    for i, path in enumerate(tqdm(paths)):
        _id = path.split('/')[-1][:-4]
        # transcribe to get speech-to-text data
        print(path)
        try:
            result = model.transcribe(path)
        except:
            print('error, removing file ',path)
            os.remove(path)
            continue
        # add results to data list
        transcriptions.extend(result['segments'])
        
        # set window (length of text chunk) and stride
        window = 1
        stride = 1  # smaller stride creates overlap
        
        data = []
        results = []
    
        with open("transcription.jsonl", "w", encoding="utf-8") as fp:
            _id = path.split('/')[-1][:-4]
            # transcribe to get speech-to-text data
            result = model.transcribe(path)
            segments = result['segments']
            for j in range(0, len(segments), stride):
                j_end = min(j+window, len(segments)-1)
                text = ''.join([x["text"] for x in segments[j:j_end]])
                start = segments[j]['start']
                end = segments[j_end]['end']
                row_id = f"{_id}-t{segments[j]['start']}"
                meta = {
                    **{
                        "id": row_id,
                        "text": text.strip(),
                        "start": start,
                        "end": end,
                        "url": f"https://youtu.be/{video_index[_id]}",
                        "name":_id,
                        "title":_id
                    }
                }
                data.append(meta)
                json.dump(meta, fp)
                fp.write('\n')
                    
        new_data = []
        
        window = 6  # number of sentences to combine
        stride = 3  # number of sentences to 'stride' over, used to create overlap
        
        for i in tqdm(range(0, len(data), stride)):
            i_end = min(len(data)-1, i+window)
            if data[i]['name'] != data[i_end]['name']:
                # in this case we skip this entry as we have start/end of two videos
                continue
            text = get_text_from_data(i, i_end, data)
            new_data.append({
                'start': data[i]['start'],
                'end': data[i_end]['end'],
                'text': text,
                'id': data[i]['id'],
                'url': data[i]['url']+'?t='+str(int(data[i]['start'])),
                "name":data[i]['name'],
                "title":data[i]['title'],
            })
            
        # loop through in batches of 64
        index = pinecone.Index(index_id)
        for j in tqdm(range(0, len(new_data), batch_size)):
            # find end position of batch (for when we hit end of data)
            j_end = min(len(new_data)-1, j+batch_size)
            # extract the metadata like text, start/end positions, etc
            batch_meta = [{
                "text": new_data[x]["text"],
                "start": new_data[x]["start"],
                "end": new_data[x]["end"],
                "url": new_data[x]["url"],
                "name": new_data[x]["name"],
                "title": new_data[x]["title"]
            } for x in range(j, j_end)]
            # extract only text to be encoded by embedding model
            batch_text = [row['text'] for row in new_data[j:j_end]]
            # create the embedding vectors
            batch_embeds = model_embed.encode(batch_text).tolist()
            # extract IDs to be attached to each embedding and metadata
            batch_ids = [row['id'] for row in new_data[j:j_end]]
            # 'upsert' (eg insert) IDs, embeddings, and metadata to index
            try:
                to_upsert = list(zip(batch_ids, batch_embeds, batch_meta))
                index.upsert(to_upsert)
            except:
                continue
        print('removing file ',path)
        os.remove(path)

https://www.youtube.com/watch?v=RETO42jnuJM @FinancialTimes
https://www.youtube.com/watch?v=yGGzimG8VMQ @FinancialTimes
https://www.youtube.com/watch?v=6cxalUYGU8g @FinancialTimes
https://www.youtube.com/watch?v=TXB7HIkQtwA @FinancialTimes
https://www.youtube.com/watch?v=Rj8qVwe_g1Y @FinancialTimes
https://www.youtube.com/watch?v=dKEY71Dpo1w @FinancialTimes
https://www.youtube.com/watch?v=hdPpsHqONEg @FinancialTimes
https://www.youtube.com/watch?v=xWIdu0B1Arc @FinancialTimes
https://www.youtube.com/watch?v=95jabRNJ6Ng @FinancialTimes
https://www.youtube.com/watch?v=6UY2HOpuTlk @FinancialTimes
https://www.youtube.com/watch?v=hW_liASsmrc @FinancialTimes
https://www.youtube.com/watch?v=DKYKT4pvYYA @FinancialTimes
https://www.youtube.com/watch?v=EQw1lajDICY @FinancialTimes
https://www.youtube.com/watch?v=jvFBl9Lv_YA @FinancialTimes
https://www.youtube.com/watch?v=uhS4n2EO7q0 @FinancialTimes
https://www.youtube.com/watch?v=nrk-8KfLY98 @FinancialTimes
https://www.youtube.com/watch?v=f-aHbL2m

  0%|          | 0/30 [00:00<?, ?it/s]


                                                                                                                     
chunk:  73%|████████████████████████████████████████████                | 4844/6595 [01:17<00:00, 2709.07it/s, now=None]

skipping file  @FinancialTimes/Capture whos looking after the children  FT Film Standpoint.mp3
skipping file  @FinancialTimes/First look at celebrity chef Marcus Samuelssons new NYC restaurant  FT Globetrotter.mp3
MoviePy - Writing audio in @FinancialTimes/How the buyout of Morrisons turned into a costly blunder  FT Due Diligence.mp3




chunk:   0%|                                                                         | 0/3246 [00:00<?, ?it/s, now=None]

chunk:   5%|██▉                                                          | 157/3246 [00:00<00:01, 1568.83it/s, now=None]

chunk:  10%|██████▏                                                      | 327/3246 [00:00<00:01, 1636.15it/s, now=None]

chunk:  15%|█████████▎                                                   | 495/3246 [00:00<00:01, 1655.37it/s, now=None]

chunk:  20%|████████████▍                                                | 661/3246 [00:00<00:01, 1612.53it/s, now=None]

chunk:  26%|███████████████▌                                             | 831/3246 [00:00<00:01, 1643.35it/s, now=None]

chunk:  31%|██████████████████▋                                          | 996/3246 [00:00<00:01, 1615.09it/s, now=None]

chunk:  36%|█████████████████████▍                                      | 1158/3246 [00:00<00:01, 1572.91it/s, now=None]

chunk:  41%|██████████

...



chunk:   0%|                                                                        | 0/39535 [00:00<?, ?it/s, now=None]

chunk:   0%|▎                                                           | 189/39535 [00:00<00:21, 1872.55it/s, now=None]

chunk:   1%|▌                                                           | 383/39535 [00:00<00:20, 1909.97it/s, now=None]

chunk:   1%|▊                                                           | 575/39535 [00:00<00:20, 1873.70it/s, now=None]

chunk:   2%|█▏                                                          | 766/39535 [00:00<00:20, 1880.03it/s, now=None]

chunk:   2%|█▍                                                          | 955/39535 [00:00<00:20, 1848.65it/s, now=None]

chunk:   3%|█▋                                                         | 1146/39535 [00:00<00:20, 1861.47it/s, now=None]

chunk:   3%|█▉                                                         | 1333/39535 [00:00<00:20, 1853.39it/s, now=None]

chunk:   4%|██▎       

chunk:  32%|██████████████████▎                                       | 12508/39535 [00:06<00:13, 1950.81it/s, now=None]

chunk:  32%|██████████████████▋                                       | 12704/39535 [00:06<00:13, 1919.85it/s, now=None]

chunk:  33%|██████████████████▉                                       | 12897/39535 [00:06<00:13, 1904.84it/s, now=None]

chunk:  33%|███████████████████▏                                      | 13088/39535 [00:06<00:13, 1900.39it/s, now=None]

chunk:  34%|███████████████████▍                                      | 13284/39535 [00:06<00:13, 1910.40it/s, now=None]

chunk:  34%|███████████████████▊                                      | 13476/39535 [00:07<00:13, 1894.31it/s, now=None]

chunk:  35%|████████████████████                                      | 13673/39535 [00:07<00:13, 1905.74it/s, now=None]

chunk:  35%|████████████████████▎                                     | 13873/39535 [00:07<00:13, 1932.69it/s, now=None]

chunk:  36%|████████████

chunk:  96%|███████████████████████████████████████████████████████▌  | 37911/39535 [00:19<00:00, 1748.15it/s, now=None]

chunk:  96%|███████████████████████████████████████████████████████▉  | 38089/39535 [00:19<00:00, 1749.55it/s, now=None]

chunk:  97%|████████████████████████████████████████████████████████▏ | 38265/39535 [00:19<00:00, 1743.69it/s, now=None]

chunk:  97%|████████████████████████████████████████████████████████▍ | 38440/39535 [00:20<00:00, 1729.70it/s, now=None]

chunk:  98%|████████████████████████████████████████████████████████▋ | 38614/39535 [00:20<00:00, 1719.11it/s, now=None]

chunk:  98%|████████████████████████████████████████████████████████▉ | 38786/39535 [00:20<00:00, 1717.55it/s, now=None]

chunk:  99%|█████████████████████████████████████████████████████████▏| 38960/39535 [00:20<00:00, 1716.90it/s, now=None]

chunk:  99%|█████████████████████████████████████████████████████████▍| 39146/39535 [00:20<00:00, 1758.94it/s, now=None]

chunk:  99%|████████████

MoviePy - Done.
MoviePy - Writing audio in @FinancialTimes/Recycling the world’s hard drive waste  FT Rethink.mp3




chunk:   0%|                                                                         | 0/3996 [00:00<?, ?it/s, now=None]

chunk:   4%|██▋                                                          | 179/3996 [00:00<00:02, 1766.57it/s, now=None]

chunk:   9%|█████▋                                                       | 375/3996 [00:00<00:01, 1860.98it/s, now=None]

chunk:  14%|████████▊                                                    | 575/3996 [00:00<00:01, 1915.99it/s, now=None]

chunk:  19%|███████████▋                                                 | 769/3996 [00:00<00:01, 1920.98it/s, now=None]

chunk:  24%|██████████████▋                                              | 962/3996 [00:00<00:01, 1883.38it/s, now=None]

chunk:  29%|█████████████████▎                                          | 1151/3996 [00:00<00:01, 1882.69it/s, now=None]

chunk:  34%|████████████████████▎                                       | 1351/3996 [00:00<00:01, 1913.49it/s, now=None]

chunk:  39%|██████████

## Query pinecone index for answer to question with video link in URL

In [60]:
index_id = "audio"
pinecone.init(
    api_key=pinecone_api_key,  # app.pinecone.io
    environment="us-west1-gcp"
)
if index_id not in pinecone.list_indexes():
    pinecone.create_index(
        index_id,
        dim,
        metric="dotproduct"
    )

index = pinecone.Index(index_id)
index.describe_index_stats()

{'dimension': 768,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 14005}},
 'total_vector_count': 14005}

In [61]:
query = "How to make a coating that lowers temperature?"

xq = model_embed.encode(query).tolist()

index.query(xq, top_k=5, include_metadata=True)

{'matches': [{'id': 'Make Our Best Thermal Paste YOURSELF!-t633.04',
              'metadata': {'end': 680.2,
                           'name': 'Make Our Best Thermal Paste YOURSELF!',
                           'start': 633.04,
                           'text': 'that you could do that is to simply take a '
                                   'drop of oil or liquid, put it between the '
                                   'two surfaces like this and eliminate the '
                                   'air and you will substantially improve the '
                                   "thermal conductivity. Nevertheless, you're "
                                   "limited to a material, the oil, that's "
                                   'about a thousand times worse than the '
                                   'thermal conductivity of the bulk '
                                   'materials. Now, if you could apply a '
                                   'sufficient amount of pressure, la